In [1]:
include("foldexity.jl")

In [13]:
pdb = readpdb("testpdb/small.pdb")
writepdb(pdb, "testpdb/small_backbone.pdb")

xyzcoords = pdb2matrix(pdb)
coordmatrix = matrix2fragments(xyzcoords, 12)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix)

println()


In [14]:
pfrag1 = coordmatrix[10]
pfrag2 = coordmatrix[31]

align_rmsd(pfrag1, pfrag2)

19.746475496483697

In [15]:
aver_rmsd, nclusts, fxity, nres, matrix = fxity_kabsh(coordmatrix)

(12.604134669018569, 37, 0.9487179487179487, 39, [0.0 15.259918127243338 … 15.673347910584875 6.248241906285712; 15.259918127243338 0.0 … 17.808255124041203 14.988237305484544; … ; 15.673347910584875 17.808255124041203 … 0.0 14.628995374500345; 6.248241906285712 14.988237305484544 … 14.628995374500345 0.0])

In [37]:
fxity, nfrags = fxpdb("testpdb/small.pdb") 
data = fxdir("testpdb/", "tmp", 4, true)  
rm("tmp")

Starting foldexity ...
8 files are collected 
Starting foldexity ...
1	testpdb/3EFM.pdb	0.827768014059754	471	4.470485886057379	569
2	testpdb/AF-Q8VCK6-F1-model_v4.pdb	0.41284403669724773	135	2.6608642385790833	327
3	testpdb/d1dcea1.pdb	0.6212121212121212	205	3.26047254437095	330
4	testpdb/d1h6ta2.pdb	0.6859903381642513	142	11.910288205705166	207
5	testpdb/d2vdfa1.pdb	0.8288288288288288	184	7.221885034927906	222
6	testpdb/polya.pdb	0.2413793103448276	14	0.364438980618678	58
7	testpdb/small.pdb	0.46808510638297873	22	2.208886731628906	47
8	testpdb/small_backbone.pdb	0.46808510638297873	22	2.208886731628906	47
